In [30]:
#open data
include("2_4.jl")
inputfilepath = "data_2-4"
inputfilename = "Data_test_5_1_2.txt"
nb_dem, nb_prod, nb_mag, nb_noeuds, matrice_S, matrice_Q, matrice_R = read_data_24(inputfilepath, inputfilename)

(5, 2, 1, 6, Array{Float64,N} where N[[15.0, 10.0]], Array{Float64,N} where N[[1.0, 0.0], [0.0, 3.0], [2.0, 1.0], [3.0, 2.0], [4.0, 4.0]], Array{Float64,N} where N[[0.0, 1.0, 1.0, 7.0, 7.0, 8.0], [1.0, 0.0, 1.0, 6.0, 5.0, 7.0], [1.0, 1.0, 0.0, 5.0, 6.0, 7.0], [7.0, 6.0, 5.0, 0.0, 1.0, 1.0], [7.0, 5.0, 6.0, 1.0, 0.0, 1.0], [8.0, 7.0, 7.0, 1.0, 1.0, 0.0]])

is_solved (generic function with 1 method)

In [40]:
using Cbc
using JuMP

# set optimizer
model = Model(Cbc.Optimizer)

# define variables
@variable(model, x[1:nb_noeuds, 1:nb_noeuds, 1:nb_mag] >= 0, Int)
@variable(model, A[1:nb_mag, 1:nb_prod, 1:nb_dem] >= 0, Int)

# define objective function
# x[i, j, k] = 1 si le livreur k traverse le chemin v(i, j) | 0 sinon
@objective(model, Min, sum(sum(sum(x[i, j, k] * matrice_R[i][j] for i in 1:nb_noeuds) for j in 1:nb_noeuds) for k in 1:nb_mag))

#Constraints

# le meme point ne peut pas etre son propre predecesseur / successeur 
for i=1:nb_noeuds
    for k=1:nb_mag
        @constraint(model, x[i, i, k] == 0)
    end
end

# chaque point i possede un seul successeur
for i=1:nb_noeuds
    for k=1:nb_mag
       @constraint(model, sum(x[i, 1:nb_noeuds, k]) == 1)
    end
end

# chaque point j possede un seul predecesseur
for j=1:nb_noeuds
    for k=1:nb_mag
        @constraint(model, sum(x[1:nb_noeuds, j, k]) == 1)
    end
end

# pas d'aller retour
for i=1:nb_noeuds
    for j=1:nb_noeuds
        for k=1:nb_mag
            @constraint(model, x[i, j, k] + x[j, i, k] <= 1)
        end
    end
end

# pas de transitions entre les magasins
for i in 1:nb_mag
    for j in 1:nb_mag
        for k in 1:nb_mag
            @constraint(model, x[i, j, k] == 0)
        end
    end 
end

# Aucun magasin ne peut livrer plus que ce qu'il a en stock
for i in 1:nb_mag
    for j in 1:nb_prod
        @constraint(model,  sum(A[i,j,k] for k in 1:nb_dem) <= matrice_S[i][j])
    end
end

# Le nombre total demandé par chaque client
for j in 1:nb_prod
    for k in 1:nb_dem
        @constraint(model,  sum(A[i,j,k] for i in 1:nb_mag ) == matrice_Q[k][j])
    end
end

# un livreur ne peut pas servir autre que ses clients
for k in 1:nb_dem
    for i in 1:nb_mag
        @constraint(model, sum(A[i, j, k] for j in 1:nb_prod) >= sum(x[k + nb_mag, l, i] for l in 1:nb_noeuds ))
        @constraint(model, sum(A[i, j, k] for j in 1:nb_prod) >= sum(x[l, k + nb_mag, i] for l in 1:nb_noeuds ))
    end
end


while !is_solved(model, x)
    optimize!(model)
end


LoadError: OptimizeNotCalled()

In [24]:
for i in 1:nb_noeuds
    for j in 1:nb_noeuds
        for k in 1:nb_mag 
           if (value(x[i,j,k]) != 0)
                println("\t le livreur $k traverse $i vers $j")
            end 
        end
    end 
end
for i in 1:nb_mag
    for j in 1:nb_prod
        for k in 1:nb_dem
            if (value(A[i,j,k])) != 0
                println("\t le client $k prend $(value(A[i,j,k])) du F$j a partir du magasin $i")
            end
        end
    end
end

	 le livreur 1 traverse 1 vers 2
	 le livreur 1 traverse 2 vers 3
	 le livreur 1 traverse 3 vers 1
	 le livreur 1 traverse 4 vers 5
	 le livreur 1 traverse 5 vers 6
	 le livreur 1 traverse 6 vers 4
	 le client 1 prend 1.0 du F1 a partir du magasin 1
	 le client 3 prend 2.0 du F1 a partir du magasin 1
	 le client 4 prend 3.0 du F1 a partir du magasin 1
	 le client 5 prend 4.0 du F1 a partir du magasin 1
	 le client 2 prend 3.0 du F2 a partir du magasin 1
	 le client 3 prend 1.0 du F2 a partir du magasin 1
	 le client 4 prend 2.0 du F2 a partir du magasin 1
	 le client 5 prend 4.0 du F2 a partir du magasin 1
